In [2]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score
from sklearn.metrics import (
    classification_report, accuracy_score, f1_score,
    precision_score, recall_score, make_scorer
)

df = pd.read_csv("/data/kangwenqq/work-acne/medicine/git/dataset.csv", encoding="utf-8")

features = [
    "Gender", "Age", "External_antibiotics",
    "TCM", "Oral_retinoids", "Oral_antibiotics", "External_retinoids",
    "Before_rank", "Duration_weeks", "Treatment_count"
]
target = "improvement_binary"
X, y = df[features], df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

param_dist_cat = {
    'iterations': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'depth': [3, 4, 5, 6, 7],
    'l2_leaf_reg': [1, 3, 5, 10],
    'bootstrap_type': ['Bayesian'],
    'bagging_temperature': [0, 0.5, 1, 2],
    'random_strength': [1, 3, 5, 10]
}

cat_model = CatBoostClassifier(verbose=0, random_state=42)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

random_search_cat = RandomizedSearchCV(
    estimator=cat_model,
    param_distributions=param_dist_cat,
    scoring='accuracy',
    n_iter=100,
    cv=kf,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search_cat.fit(X_train, y_train)

print("\n=== CatBoost RandomizedSearchCV Results ===")
print("Best Params:", random_search_cat.best_params_)
print("Best CV Accuracy:", random_search_cat.best_score_)

best_cat_model = random_search_cat.best_estimator_
y_pred = best_cat_model.predict(X_test)

print("\n=== Test Evaluation ===")
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

cv_results = random_search_cat.cv_results_
best_index = random_search_cat.best_index_

mean_cv_score = cv_results['mean_test_score'][best_index]
std_cv_score = cv_results['std_test_score'][best_index]
print(f"\nCatBoost CV Accuracy: {mean_cv_score:.4f} ± {std_cv_score:.4f}")

# F1-score
f1_cv_scores = cross_val_score(best_cat_model, X, y, cv=10, scoring='f1_macro')
# Precision
precision_cv_scores = cross_val_score(
    best_cat_model, X, y, cv=10,
    scoring=make_scorer(precision_score, average='macro', zero_division=1)
)
# Recall
recall_cv_scores = cross_val_score(best_cat_model, X, y, cv=10, scoring='recall_macro')

print(f"\n=== CatBoost (10-fold CV) Metrics ===")
print(f"  F1-score :  {f1_cv_scores.mean():.4f} ± {f1_cv_scores.std():.4f}")
print(f"  Precision:  {precision_cv_scores.mean():.4f} ± {precision_cv_scores.std():.4f}")
print(f"  Recall   :  {recall_cv_scores.mean():.4f} ± {recall_cv_scores.std():.4f}")


Fitting 10 folds for each of 100 candidates, totalling 1000 fits

=== CatBoost RandomizedSearchCV Results ===
Best Params: {'random_strength': 5, 'learning_rate': 0.01, 'l2_leaf_reg': 10, 'iterations': 500, 'depth': 6, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0}
Best CV Accuracy: 0.8328947368421051

=== Test Evaluation ===
Test Accuracy: 0.86
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.46      0.63        13
           1       0.84      1.00      0.91        37

    accuracy                           0.86        50
   macro avg       0.92      0.73      0.77        50
weighted avg       0.88      0.86      0.84        50


CatBoost CV Accuracy: 0.8329 ± 0.1037

=== CatBoost (10-fold CV) Metrics ===
  F1-score :  0.7086 ± 0.1353
  Precision:  0.9121 ± 0.0284
  Recall   :  0.6905 ± 0.1103
